# 02. Logistic Regression with TF

<p style="text-align: right;">
rayleigh@dplus.company<br>
Your name :
</p>

### 텐서플로 실습시간 그 세번째!<br>

#### 선행지식
1. 모델링을 한다는 것에 대한 이해.
2. Supervised Learning 중 Classification에 대한 기본 지식.
3. 로지스틱 리그레션에 대한 기본 구조와 개념
4. 앞서 진행한 2번의 실습에 대한 추억(00. Hello World with TF, 01. Linear Regression with TF)

#### 실습목표
1. Logistic Regression의 구조를 Graph로 그려낼 수 있다.
2. 그려낸 Graph를 텐서플로우를 이용해 코딩할 수 있다.
3. (Optional) Encoding이 무엇인지 이해한다. --> Dummy Variable, Onehot-encoding 등의 필요성을 이해한다.
4. (Optional) Logistic Regression 과 Softmax Regression이 사실상 같은 내용임을 알 수 있다.

#### 사용데이터.

01. Binary Classification : https://stats.idre.ucla.edu/stat/data/binary.csv
02. Multinomial Classification : http://yann.lecun.com/exdb/mnist/

## 00. 라이브러리 불러오기

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import time
import os

# 현재 작업 디렉토리 확인
os.getcwd()

## 01. 데이터 불러오기 & 문제정의 & 전처리

In [ ]:
'''
pd.read_csv 뒤에, 정확한 파일 url을 넣어도 동작합니다.
'''
data = pd.read_csv("https://stats.idre.ucla.edu/stat/data/binary.csv")
data.head(n=10)

In [ ]:
# rank는 더미가 필요
dummy = pd.get_dummies(data['rank'])
temp = data.drop('rank', axis=1)
data = pd.concat([temp, dummy], axis=1)
data.head(n=10)


#### 데이터 설명.

A researcher is interested in how variables, such as GRE (Graduate Record Exam scores), GPA (grade point average) and prestige of the undergraduate institution, effect admission into graduate school. The response variable, admit/don’t admit, is a binary variable.

나는 대학원 지원자.<br>
내가 합격할 가능성은 어느정도나 될까..?
그리고 그것을 어떻게 확인할까..?

In [ ]:
# Pandas DataFrame to Numpy Array
data_array = data.values
print(data_array[:5], data_array.shape)

In [ ]:
# Shuffle!
np.random.shuffle(data_array)
print(data_array)

In [ ]:
# Training set / Test set Split
train, test = data_array[:300], data_array[300:]
X_train, Y_train = train[:, 1:], train[:, :1]  # shape 확인 꼭 해볼 것.
X_test, Y_test = test[:, 1:], test[:, :1]

In [ ]:
print(X_train.shape, Y_train.shape)
print(X_train, Y_train)

## 02. 손으로 그려보는 Logistic Regression



*직접, 손으로 구조를 설계 해보는 것은 중요하다.*<br>

여러분이 텐서플로우로 '그래프'를 코딩하기에 최대한 편한 설계도를 그리면 된다.<br>
꼭 Data flow Graph, Computation Grpah, Probabilistic Graph 등의 표현을 따를 필요는 없다.(물론, 익히면 좋다.)<br>


### 예시.  여러분이 직접 그려보기 전에는 실행 시키지 말기!

[로지스틱01](img/Logistic_1.PNG)
[로지스틱02](img/Logistic_2.PNG)

### 반드시 차원을 고려 할 것!

X의 shape = [ 채워넣기! , 채워넣기! ]

Y의 shape = [ 채워넣기! , 채워넣기! ]

W의 shape = [ 채워넣기! , 채워넣기! ]

bias의 shape = [ 채워넣기! , 채워넣기! ]

연산순서는?

## 03. Make a Graph with TF

In [ ]:
'''
여러분이 직접 스크립트를 짜야 합니다.
조금 힌트는 드립니다..!
'''
# Graph Clear & Make your Graph reproducible
tf.reset_default_graph()
tf.set_random_seed(2017)

# Hyper-Parameters & Option
learning_rate = 0.01

# Placeholders
X = tf.placeholder(dtype=tf.float32, shape=[None, 6], name='X_input')
Y = tf.placeholder(dtype=tf.float32, shape=[None, 1], name='Y_input')

# Weight & Bias
W = tf.Variable(tf.random_normal([6, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name="bias")

# Hypothesis
logits = tf.add(tf.matmul(X, W), b, name='logits')
Y_prob = tf.sigmoid(logits, name='Y_probability')

# Classification & Accuracy
Y_pred = tf.cast(Y_prob > 0.5, dtype=tf.float32, name='Y_predicted')
accuracy = tf.reduce_mean(tf.cast(tf.equal(Y_pred, Y), dtype=tf.float32), name='Accuracy')

# Cost(loss) function & Optimizer
# cost = -1 * tf.reduce_mean(Y * tf.log(Y_prob) + (1 - Y) * tf.log(1 - Y_prob))
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


## 04. Train! Session!

In [ ]:
sess = tf.Session()
tf.set_random_seed(2017)
sess.run(tf.global_variables_initializer())

learning_rate = 0.01

for i in range(10000):
    # 학습 진행
    sess.run(optimizer, feed_dict={X: X_train, Y: Y_train})
    
    if (i % 100 == 0):
        # 학습 상황 디스플레이
        tr_l, tr_a = sess.run([cost, accuracy], feed_dict={X: X_train, Y: Y_train})        
        print( "{}th epoch, training accuracy: {:.2f}%, training loss: {:.2f}".format(i, tr_a * 100, tr_l))

print("Training Complete")        
test_accuracy = sess.run(accuracy, feed_dict={X: X_test, Y: Y_test})
print("Test Accuracy: {:.2f}%".format(test_accuracy * 100))
print("Admit Rate: {:.2f}%".format(data_array[:, 0].mean() * 100))
print(sess.run([W[:, 0], b]))

sess.close()

## 이제는 MNIST를 분류해보자!

In [ ]:
tf.reset_default_graph()
%reset

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./MNIST_data/", one_hot=True)
'''
텐서플로우의 input_data를 이용하여 MNIST를 다루면
데이터를 처리하기가 매우 편리해진다.
'''


In [ ]:
'''
mnist 시각화 함수
'''
def mnist_plot(i):
    pixels = mnist.test.images[i].reshape((28, 28))
    a = mnist.test.labels.tolist()
    plt.title('Label: {}'.format(a[i].index(1)))
    plt.imshow(pixels, cmap='gray')
    return plt.show()

In [ ]:
'''
0 ~ 9999 사이의 숫자를 넣어서 확인해보자.
'''
mnist_plot(1000)

### 00. 문제를 정의하고, 데이터를 확인해보자.

옛날 옛적, 호랑이가 딱 담배 끊을즈음 시기에..

우리는 우체국에서 일하는 직원이었다..

나는 하루 종일 우편번호를 읽고 각 동별로 분류작업을 해야 했다..

편지는 하루 1000여 통 이상 도착하고.. 나는 하루 4시간은 꼬박 분류만 해야한다..

지친다...

#### 문제상황은? :

In [ ]:
'''
트레이닝 데이터로 활용할 55000개 이미지, 테스트용 10000개 이미지
28*28 사이즈의 이미지가 그냥 주욱 784칸 짜리 어레이에 담겨있다.
레이블은 이미 one-hot encoding이 되어 있다.

'''
print(mnist.train.images.shape, mnist.train.labels.shape)
print(mnist.test.images.shape, mnist.test.labels.shape)

In [ ]:
mnist.train.images[0]

In [ ]:
mnist.train.labels[0]

### 01. 손으로 설계해보자!

위의 문제와는 다르다.
Logistic Regression의 sigmoid 함수는 binary classification 용이다.<br>
multinomial classification을 위해서는 Softmax를 사용해야 하는데,<br>
무서워하지 말자. sigmoid의 확장판일 뿐!<br>
Multinomial Logistic Regression을 간단히 Softmax Regression을 사용할 뿐!<br>

인터넷에는 아래와 같은 이미지도 돌아다닌다.
![이미지](https://image.slidesharecdn.com/deeplearningintheano-160311150524/95/deep-learning-in-theano-10-638.jpg?cb=1457708806)

#### 하지만 손으로 직접 그려보자. 자기가 이해한 방식대로.



### 02. 텐서서플로우로 그래프를 만들자!

In [ ]:
# Graph Clear & Make your Graph reproducible
tf.reset_default_graph()
tf.set_random_seed(2017)

# Hyper parameters & Option
learning_rate = 0.01

# Place Holders
X = tf.placeholder(dtype=tf.float32, shape=[None, 784], name='X_input')
Y = tf.placeholder(dtype=tf.float32, shape=[None, 10], name='Y_input')

# Weights & bias
W = tf.Variable(tf.random_normal([784, 10]), name="weight")
b = tf.Variable(tf.random_normal([10]), name="bias")

# Hypothesis
logits = tf.add(tf.matmul(X, W), b, name='logits')
Y_prob = tf.nn.softmax(logits, name='Y_distribution')

# Accuracy
n_of_correct = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy =  tf.reduce_mean(tf.cast(n_of_correct, tf.float32))

# Cost(loss) function & Optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)



### 03. 세션을 만들고, 학습시키자!

In [ ]:
sess = tf.Session()
tf.set_random_seed(2017)
sess.run(tf.global_variables_initializer())

learning_rate = 0.01

print(mnist.train.num_examples)
for i in range(100):
    '''
    갯수가 너무 많아서 배치로 끊어서 학습
    '''
    n_of_batches = int(mnist.train.num_examples / 100)
    for batch in range(n_of_batches):
        X_batch, Y_batch = mnist.train.next_batch(100)
        # 학습 진행
        sess.run(optimizer, feed_dict={X: X_batch, Y: Y_batch})

    # 학습 상황 디스플레이
    tr_a = sess.run(accuracy, feed_dict={X: X_batch, Y: Y_batch})
    print("{}th epoch, training accuracy: {:.3f}%".format(i, tr_a * 100))
        
print("Training Complete")        
test_accuracy = sess.run(accuracy,
                         feed_dict={X: mnist.test.images, Y: mnist.test.labels})
print("Test Accuracy: {:.2f}%".format(test_accuracy * 100))
# print(sess.run([W[:, 0], b]))

#### batch & epochs

In [ ]:
sess = tf.Session()
tf.set_random_seed(2017)
sess.run(tf.global_variables_initializer())

# batch_size를 바꿔가며 반복 실행해보자.
training_epochs = 10
batch_size = 100
learning_rate = 0.01

for i in range(training_epochs + 1):
    # batch의 횟수 계산
    n_of_batches = int(mnist.train.num_examples / batch_size)
    
    for batch in range(n_of_batches):
        X_batch, Y_batch = mnist.train.next_batch(batch_size)
        # 학습 진행
        sess.run(optimizer, feed_dict={X: X_batch, Y: Y_batch})
        
    # 학습 상황 디스플레이
    tr_a = sess.run(accuracy, feed_dict={X: X_batch, Y: Y_batch})
    print("{}th epoch, training accuracy: {:.3f}%".format(i, tr_a * 100))
        
print("Training Complete")        
test_accuracy = sess.run(accuracy,
                         feed_dict={X: mnist.test.images, Y: mnist.test.labels})
print("Test Accuracy: {:.2f}%".format(test_accuracy * 100))
# print(sess.run([W[:, 0], b]))

### 성능을 한 번 확인해 보자. 제대로 되고 있는걸까?

In [ ]:
n = int(np.random.uniform(0, 9999, 1))
n = 217
Predicted_distribution = sess.run(Y_prob, feed_dict={X: [mnist.test.images[n]]})
Predicted_distribution = Predicted_distribution[0]
pd_dict = {i: '%.2f' % prob for i, prob in enumerate(Predicted_distribution)}

print(n)
print(pd_dict)
mnist_plot(n)


### 실습

In [ ]:
'''
TensorFlow 그래프를 전체를 직접 작성해 보자
'''






In [ ]:
'''
학습하는 세션 코드 전체를 직접 작성해 보자
'''






# must Summary!